<a href="https://colab.research.google.com/github/iciromaco/AliceLesson/blob/master/5CNN/CIFAR10PyTorch002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color=red>**【共有中に付き編集不可】**　改変したい場合はコピーを作成してください　PR,IAI</font>

In [ ]:
# @title
# 動作環境の確認

import datetime
import pytz

def 時刻表示(phase):
    japan_tz = pytz.timezone('Asia/Tokyo')
    current_japan_time = datetime.datetime.now(japan_tz)
    formatted_time = current_japan_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
    print(phase,formatted_time)
    return formatted_time
時刻表示('開始時刻')

!pip -q install deep-translator requests
!pip -q install icrawler
!pip -q install icrawler deep-translator nltk torch transformers pillow

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
if device.type != 'cpu':
  print(torch.cuda.get_device_name())
  print(torch.cuda.get_device_capability())

# 画像の収集
画像を認識するためには、まず事前にたくさんの事例をAIに機械学習させておく必要がありますが、
大量の画像を自分で用意するのは大変なので、プログラムで収集することにします。

以下、重要な指示は<font color='blue'>青字</font>，特に重要な指示は<font color='red'>赤字</font>にしてありますので、見落とさないように演習を進めていってください。

# 1. Google Drive をマウント
Google Colaboratory は一定時間経過すると作業内容が削除されてしまいます。消したくないデータや後日再利用したいデータは Google Drive に保存するようにプログラムすることで、残すことができます。


1. <font color='blue'>この説明の下のセルを実行する
2. アクセス許可の確認ダイアログが出る。「Googleドライブに接続」をクリック。
3. アカウントを選択する。
4. アクセスリクエストのダイアログに変わる。一番下までスクロールし「許可」をクリック。
5. 左のファイル一覧エリアで「更新アイコン」（回転マーク）をクリック（エリアが表示されていないなら「ファイルアイコン」（フォルダ型）をクリック）</font>


<img width="390" alt="googlecolab" src="https://user-images.githubusercontent.com/5820803/94802343-739cff00-0422-11eb-8c0d-affa919f8e58.png">

　<font color=blue>次のセルを実行してください</font>




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2. 画像の収集と選別

自分で収集した画像を使って独自のデータセットを作り、先と同様の実験をやってみましょう。

## 2-1 画像の収集

画像を収集するために必要なライブラリを準備します。  <font color='blue'>次のセルを実行してください</font>

In [ ]:
#@title [１] ライブラリ準備
# ==============================================================================
# 1. ライブラリ準備
# ==============================================================================

import sys
import os
import glob
import cv2
import shutil
import logging
import time
import requests

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
from deep_translator import GoogleTranslator
from icrawler.builtin import BingImageCrawler
from nltk.corpus import wordnet
import nltk
from transformers import CLIPProcessor, CLIPModel

# from icrawler.builtin import GoogleImageCrawler
import warnings
import datetime
warnings.simplefilter('ignore')

ViewSIZE = 128 # 表示用画像サイズ高さと幅
SIZE = 32 # モデルに入力する際の画像サイズ

機械学習の際に画像のサイズをすべて同じに揃えます。ViewSIZEは表示用の画像サイズ、SIZEは学習・認識時のサイズです。  
CIFAR10に合わせるためにSIZE = 32としています。

次に、収集する画像のカテゴリを決めましょう。



--
CIFAR-10 のカテゴリは
- airplane (飛行機)
- automobile (自動車）
- bird (鳥類）
- cat （ネコ)
- deer (シカ）
- dog （イヌ）
- frog （カエル）
- horse（ウマ）
- ship（船舶）
- truck（トラック）

でした。比較したいので、<font color=red>**ここに含まれるの中から「ネコ」とネコ以外のカテゴリを１つ**</font>、さらに、ここに<font color=red>**含まれないカテゴリを２つ**</font>，計４つのカテゴリを考えましょう。
以下では、CIFAR10のカテゴリから「ネコ」と「イヌ」、追加カテゴリとして「カメ」「スクータ」を選んだものとして説明していきます。

## **<font color=red>確認 実験に使う４つのカテゴリ</font>**
・「ネコ」を４つのうちの一つとして必ず含める．  
・CIFAR-10の10カテゴリから，ネコ以外に一つ選ぶ．  
・上に含まれないカテゴリを２つ，計４カテゴリを考える．

下のセルのカテゴリ名と検索用の同義語リストを自分で選んだものに書き換えて実行してください。  
書いてあるものから選んでも構いません。

  <font color='blue'>次のセルを実行してください</font>

In [ ]:
#@title  [２]  設定 & 収集カテゴリ定義
# ==============================================================================
# 2. 設定 & AIクラス定義
# ==============================================================================
# 収集したいカテゴリ  （次の行は例です。'ネコ'以外を書き換えてください）
myclasses = ['ネコ','イヌ','灯台','スクータ']

# 保存先設定
max_num = 300          # 収集目標
TMPFOLDER = 'drive/MyDrive/tmp/'

# オプション設定
USE_CLIP_FILTER = True   # CLIP選別を使うか
CLIP_THRESHOLD = 0.25    # このスコアを下回ったら即捨てる（0.2~0.3推奨）


ネット上で公開されている画像を収集するためのライブラリ、icrawler を使って画像を収集します。 今回は Google から画像を拾ってくることにします。

次のセルは、上で選んだカテゴリの画像をそれぞれ最大３００枚まで収集して、一旦 Google Drive のマイドライブ内の 'tmp'という名のフォルダに保存するプログラムです。tmpフォルダ内にカテゴリごとのサブフォルダができます。（ "drive/MyDrive" が Google Driveのマイドライブを表しています）

次のセルを実行 　　（定義だけなので何も起こりません）

In [ ]:
#@title 画像収集エージェントの定義
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')

# ログ抑制
logging.getLogger('icrawler').setLevel(logging.INFO)
logging.getLogger('transformers').setLevel(logging.ERROR)

# ==============================================================================
# 3. クラス定義
# ==============================================================================
class StrategyPlanner:
    """検索戦略を練るクラス：単語を品種や特徴に分解する"""
    def __init__(self):
        self.to_en = GoogleTranslator(source='auto', target='en')
        self.to_ja = GoogleTranslator(source='en', target='ja')

    def get_subcategories(self, category):
        """
        WordNetを使って「下位概念（Hyponyms）」を取得し、検索語を増やす
        例: Dog -> Poodle, Corgi, Pug...
        """
        print(f"🔍 '{category}' の種類(サブカテゴリ)を調査中...")
        sub_words = set()
        # 元の単語も入れる
        sub_words.add(category)

        try:
            en_word = self.to_en.translate(category)
            synsets = wordnet.synsets(en_word)

            for syn in synsets:
                # 下位語（種類）を取得
                for hyponym in syn.hyponyms():
                    name = hyponym.name().split('.')[0].replace('_', ' ')
                    sub_words.add(name)

            # 翻訳して日本語リストも作る（確認用）
            print(f"   -> {len(sub_words)} 種類の関連ワードを発見")
        except:
            pass

        return list(sub_words)

class ImageFilterAI:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🧠 [CLIP] モデルロード中... ({self.device})")
        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.translator = GoogleTranslator(source='auto', target='en')

    def filter(self, folder_path, category_text, negative_words, threshold):
        """
        絶対評価フィルタ: 指定した閾値(threshold)よりスコアが低い画像は全て捨てる
        """
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png'))]
        if not image_files: return

        print(f"🧹 [CLIP] 画質・関連度判定中: '{category_text}' ({len(image_files)}枚)")

        pos_text = self.translator.translate(category_text)
        neg_texts = [self.translator.translate(nc) for nc in negative_words]

        # 判定ラベル
        # 正解 vs ノイズ(排除ワード + 一般的なゴミ)
        labels = [f"a photo of a {pos_text}"]
        labels += [f"a photo of a {nt}" for nt in neg_texts]
        labels += ["text", "watermark", "meme", "screenshot", "illustration", "toy", "statue"]

        print(f"   (合格ライン: スコア {threshold} 以上)")

        deleted_count = 0
        kept_count = 0

        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            try:
                image = Image.open(img_path)
                inputs = self.processor(text=labels, images=image, return_tensors="pt", padding=True).to(self.device)
                outputs = self.model(**inputs)

                # 確率分布
                probs = outputs.logits_per_image.softmax(dim=1)[0]

                # 正解ラベル(index 0)のスコア
                positive_score = probs[0].item()
                top_idx = probs.argmax().item()

                # 判定ロジック:
                # 1. 一番高い確率が「正解」であること
                # 2. かつ、その確信度が「閾値」を超えていること
                if top_idx == 0 and positive_score >= threshold:
                    kept_count += 1
                else:
                    os.remove(img_path)
                    deleted_count += 1

            except Exception:
                os.remove(img_path)

        print(f"   -> ✅ 残った画像: {kept_count}枚 / 🗑️ 削除: {deleted_count}枚")

# [3] 検索語の決定

検索したいものを見つけるためのキーワードと排除したいものを表すキーワードを決めます。

## 検索キーワードの修正
検索は英語を使った方が多くヒットするので、まず日本語を英語に変換し、類義語をAIに提案させます。

語のニュアンスが日本語と英語とでずれている場合があります。例えば、「スクータ」は英語では「scooter」ですが、英語の「scooter」には海上スクータが含まれます。

スクータ　→　scooter　→ sea scooter, motor scooter, water scooter

AIが検索語の候補を提案してきますので、不要なものは削除してください。

→　スクータ, scooter, motor scooter

## 検索キーワードの追加
もっとヒットする可能性のある語があれば追加してください。

→　スクータ, scooter, motor scooter,ホンダスクータ、ヤマハスクータ

## 排除キーワード

検索画像には望ましくないものもたくさん含まれますので、これは含んでほしくないというものを聞いてきますので答えてください。

例）キックボード、人物、ショップ

In [ ]:
#@title  [３]  検索語の決定
# ==============================================================================
# 3. 【重要】フェーズ1：人間によるキーワードの確定
# ==============================================================================
時刻表示('検索語選択の開始')
planner = StrategyPlanner()
search_plan = {}

print("\n" + "="*60)
print("📝 STEP 1: 検索キーワードの拡張（ドリルダウン）")
print("数を稼ぐために、具体的な種類（品種など）をリストアップします。")
print("="*60 + "\n")

for categ in myclasses:
    print(f"--- カテゴリ: 【 {categ} 】 ---")

    # AIによる種類の展開
    sub_categories = planner.get_subcategories(categ)

    # 候補が多すぎると入力しづらいので、主要なもの+自由記述にする
    print(f"🤖 自動提案された種類の例 (上位表示):")
    print(f"{', '.join(sub_categories[:10])} ...など計{len(sub_categories)}個")

    print("\n👇 【重要】検索に使うキーワードを入力してください。")
    print("   目標300枚なら、最低でも10〜20個の単語が必要です。")
    print("   (入力例: 柴犬, ゴールデンレトリバー, 三毛猫, ペルシャ猫, ...)")
    print("   ※ 空欄にすると、AIが提案した全リスト（英語含む）を使って全力で集めます。")

    user_input = input(f">>> ")

    if user_input.strip():
        # ユーザー入力を採用
        keywords = [w.strip() for w in user_input.replace('、', ',').split(',') if w.strip()]
    else:
        # AI提案を全採用（数を稼ぐため）
        # ただし、元のカテゴリ名(日本語)も先頭に追加しておく
        keywords = [categ] + sub_categories

    # 排除ワード設定
    print(f"🗑️ 排除したいもの (例: イラスト, おもちゃ):")
    neg_input = input(f">>> ")
    neg_words = [w.strip() for w in neg_input.replace('、', ',').split(',') if w.strip()]

    search_plan[categ] = {'keywords': keywords, 'neg': neg_words}
    print(f"✅ 確定: {len(keywords)}種類のキーワードで検索します\n")

時刻表示('検索語決定完了')


# [4] 自動収集と自動選別
Bing検索で指定されたキーワードで画像を集め、画像を認識してキーワードに合っていないものや排除対象が写っているものを取り除きます。

In [ ]:
#@title  [４]  自動収集と自動選別
# ==============================================================================
# 4. フェーズ2：全自動実行（収集 ＋ CLIP選別）
# ==============================================================================
時刻表示('収集開始')
print("\n" + "="*60)
print("🚀 収集 & 高精度フィルタリング実行")
print("="*60 + "\n")

clip_filter = ImageFilterAI() if USE_CLIP_FILTER else None

for categ, plan in search_plan.items():
    print(f"\n📂 カテゴリ処理中: {categ}")

    keywords = plan['keywords']
    neg_list = plan['neg']

    dir_path = os.path.join(TMPFOLDER, categ)
    os.makedirs(dir_path, exist_ok=True)

    crawler = BingImageCrawler(storage={"root_dir": dir_path}, downloader_threads=4)

    # 目標枚数チェック用
    total_collected = 0

    # --- A. キーワードごとのループ収集 ---
    # リストにある単語を1つずつ検索して、少しずつ集める
    print(f"   🏃 {len(keywords)}種類のキーワードで順次検索します...")

    neg_query = " ".join([f"-{n}" for n in neg_list])

    for key in keywords:
        # 現在の枚数チェック
        current_files = len(os.listdir(dir_path))
        if current_files >= max_num * 1.5: # フィルタで減る分を見越して1.5倍まで集める
            print(f"   🛑 目標({max_num}枚)に対し、{current_files}枚集まったため収集を終了します。")
            break

        # 1つのキーワードにつき、最大50枚程度を狙う（欲張らない方がエラーが出ない）
        # クエリ: "Golden Retriever dog -illustration ..."
        full_query = f"{key} {categ} {neg_query} -illustration" # 元カテゴリ名も足して文脈を強化

        # ログがうるさいのでprintで進捗表示
        print(f"      🔍 Searching: {full_query} (現在: {current_files}枚)")

        try:
            crawler.crawl(keyword=full_query, filters={'type':'photo'}, max_num=50, overwrite=False)
        except:
            pass

        # サーバー負荷軽減
        time.sleep(1)

    # --- B. CLIP選別 (絶対評価) ---
    if clip_filter:
        clip_filter.filter(dir_path, categ, neg_list, CLIP_THRESHOLD)

    final_count = len(os.listdir(dir_path))
    print(f"🎉 カテゴリ '{categ}' 完了: 最終的に {final_count} 枚を保存しました。")
時刻表示('収集完了')

ERRORが出るのはリンク切れ(404)やアクセス制限(403)であり、プログラムのエラーではないので気にしなくても構いません。

## 2-2 画像の選別

![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig001.png)　　　　　　 ![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig002.png)   

![Google Drive](https://github.com/aquapathos/pictures/blob/gh-pages/BasicAIFig003.png?raw=true)

1. <font color='blue'>ブラウザで新しいタブをつくり、[ドライブ](https://drive.google.com/) を開きます。
2. **マイドライブ**の中の**tmp**フォルダを開いてみてください。 自分が決めたカテゴリのフォルダが並んでいるはずです。
3.「ネコ」を例として説明します。**ネコ** フォルダを開きます。
4. リストレイアウトからグリッドレイアウトに切り替えて不適だと思われる画像を削除してください。
5. 同様にして、ほかのカテゴリ（例ではイヌ、灯台、スクータ）についても不要画像を削除してください。</font>

不適なのは、次のような画像です。
- ネコが写っていない画像。写っていても小さすぎる画像
- 主要被写体がネコではない画像、つまりネコ以外のものが目立っている画像
- 文字が目立つ画像。できれば文字は入っていないことが望ましいですが、目立たなければOK
- デフォルメされたぬいぐるみやイラスト。
- GIF 動画
- 同一の写真

パッと見て、これは「ネコ」が主題の写真だと感じられるものを残して、そうでないもの、他のモノが多くの面積を占めているものは削除してください。(厳密なルールはありません。あなた自身がこれは何かと問われて「ネコ」と答える写真を選べばいいです。あなたの選んだものが教師データとなります。)

# 2-3　画像のリサイズ

何度も使うので関数を定義しておきます。  <font color='blue'>次のセルを実行してください。</font>　　（コードは非表示になっていますが、実行する必要があります。）

In [ ]:
#@title リサイズ関数の定義
def resize(foldername, size=ViewSIZE):
    imgnames = glob.glob(foldername+"/*") # 画像ファイル名のリスト
    images = []
    for imgname in imgnames:
        img = cv2.cvtColor(np.array(Image.open(imgname).convert('RGBA')),cv2.COLOR_RGBA2RGB)
        height = img.shape[0]
        width = img.shape[1]
        if height > width :
            m = (height - width)//2
            img = img[m:m+width]
        else:
            m = (width - height)//2
            img = img[:,m:m+height]
        img = cv2.resize(img, (size,size))
        images.append(img)
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        cv2.imwrite(imgname,img)
    return images

さっそくリサイズしてみましょう。　　<font color='blue'>次のセルを実行してください。</font>




In [ ]:
# @title
input('不適切画像の削除は終了したら Enterを押してください')
# リサイズを実行
images = []
for i,categ in enumerate(myclasses):
  images.append(resize(TMPFOLDER+categ))
  print(f"{categ} - {len(images[i])} 枚の画像がリサイズされました")
images = np.array(images, dtype=object) ## numpy 仕様変更あり 2024

表示用関数を定義し、表示して確認してみます。　　<font color='blue'>次のセルを実行してください。</font>　コードは非表示にしてあります。

In [ ]:
#@title 表示用関数の定義
import math
# start番からnpic枚表示する関数を定義
plt.rcParams['figure.figsize'] = (12.0, 7.0)
def showimg(images, start = 0, npic = 48):
    n = npic if len(images) >= start+npic else len(images) - start
    plt.figure(figsize=(8,7.5*(math.ceil(n/8))/6),dpi=150)
    i = 0
    while True:
        if i < n :
            plt.subplot((n-1)//8+1,8,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(images[start+i])
            plt.title("{}".format(start+i))
            i += 1
        else:
            break

さっそく表示させてみましょう。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
for i in range(len(images)):
  showimg(images = images[i],start=0,npic=8) # 各カテゴリ16枚ずつ表示

## 2-4 学習用画像データの保存と読み込み
python にはメモリ上に置かれた変数の値をそっくりそのままファイルとして保存する関数が用意されていますので、その関数を使って変数の中身を Google ドライブに保存したり、読み出したりできるようにしておきます。　　<font color='blue'>次のセルを実行してください。</font>


In [ ]:
#@title
import pickle
import os

# 画像データを pickle 形式で保存
def storeCategoryImages(cat, fname, folder = "."):
    os.makedirs(folder, exist_ok=True)
    f = open(folder+"/"+fname,'wb')
    pickle.dump(cat,f)
    f.close

# pickle 形式で保存された画像データの読み込み
def loadCategoryImages(fname, folder = "."):
    f = open(folder+"/"+fname,'rb')
    cat = pickle.load(f)
    f.close
    return cat

さっそくリサイズした画像をpickle 形式で保存しましょう。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
GFOLDER = "drive/MyDrive/CIFAR10"  # データ保存用のフォルダ名
for i,categ in enumerate(myclasses):
  print(i,categ)
  storeCategoryImages(images[i],f"{categ}.pkl",folder=GFOLDER)

今度は逆に pickle 形式で保存されたデータを変数に読み込み、正しく復元できるか確認しておきましょう。一つ目のカテゴリだけ試してみます。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
testimg = loadCategoryImages("ネコ.pkl", folder=GFOLDER)
showimg(images = testimg,start = 4,npic=8)

# CIFAR10PyTorch003 に進んでください。

------